In [206]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from pycaret.time_series import *
from sktime.forecasting.model_selection import SlidingWindowSplitter
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
warnings.filterwarnings('ignore')

In [2]:
%pip install 'pycaret[full]'

  Using cached shap-0.44.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (24 kB)
  Using cached interpret-0.5.1-py3-none-any.whl.metadata (1.1 kB)
  Using cached umap-learn-0.5.5.tar.gz (90 kB)
  Preparing metadata (setup.py) ... done
  Using cached PyYAML-5.3.1.tar.gz (269 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [51 lines of output]
      /opt/homebrew/lib/python3.11/site-packages/setuptools/config/setupcfg.py:293: _DeprecatedConfig: Deprecated config in `setup.cfg`
      !!
      
              ********************************************************************************
              The license_file parameter is deprecated, use license_files instead.
      
              This deprecation is overdue, please update your project and remove deprecated
              calls to avoid build errors in the future.
      
              See https://setuptools.pypa.io/en/l

In [7]:
%pip install 'packaging==21.3'

  Using cached packaging-21.3-py3-none-any.whl.metadata (15 kB)
Using cached packaging-21.3-py3-none-any.whl (40 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 23.2
    Uninstalling packaging-23.2:
      Successfully uninstalled packaging-23.2
Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install --upgrade 'packaging'
%pip install --upgrade 'pycaret'

  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
Using cached packaging-23.2-py3-none-any.whl (53 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [299]:
df = pd.read_csv("train.csv")
df = df.set_index('DATETIMEDATA')
df

,PM25,O3,WS,TEMP,RH,WD
DATETIMEDATA,,,,,,
2024-01-01 00:00:00,16.5,16.000000,1,27.0,79.0,115.0
2024-01-01 01:00:00,33.6,11.326513,1,27.0,82.0,97.0
2024-01-01 02:00:00,34.2,11.326513,1,26.0,84.0,96.0
2024-01-01 03:00:00,20.1,11.326513,1,26.0,82.0,108.0
2024-01-01 04:00:00,16.5,1.000000,1,27.0,79.0,106.0
...,...,...,...,...,...,...
2024-02-28 19:00:00,23.6,11.326513,1,29.0,64.0,184.0
2024-02-28 20:00:00,25.2,11.326513,1,29.0,64.0,183.0
2024-02-28 21:00:00,23.2,1.000000,1,29.0,64.0,159.0


In [300]:
import numpy as np
from sktime.forecasting.model_selection import SlidingWindowSplitter
ts = np.arange(10)
splitter = SlidingWindowSplitter(fh=np.arange(1,3), window_length=3, step_length=1)
list(splitter.split(ts))

[(array([0, 1, 2]), array([3, 4])),
 (array([1, 2, 3]), array([4, 5])),
 (array([2, 3, 4]), array([5, 6])),
 (array([3, 4, 5]), array([6, 7])),
 (array([4, 5, 6]), array([7, 8])),
 (array([5, 6, 7]), array([8, 9]))]

In [301]:
df.size

8496

In [305]:
from sklearn.model_selection import TimeSeriesSplit
from pycaret.time_series import TSForecastingExperiment
from sktime.forecasting.model_selection import SlidingWindowSplitter
exp = TSForecastingExperiment()

exp.setup(data = df, target='PM25' ,coverage=0.90, fold_strategy=SlidingWindowSplitter(fh=np.arange(1,23), window_length=130, step_length=130)) # using the past 6 months data to make prediction for the next month and moving half a year forward. 

,Description,Value
0,session_id,1507
1,Target,PM25
2,Approach,Univariate
3,Exogenous Variables,Present
4,Original data shape,"(1416, 6)"
5,Transformed data shape,"(1416, 6)"
6,Transformed train set shape,"(1394, 6)"
7,Transformed test set shape,"(22, 6)"
8,Rows with missing values,0.0%
9,Fold Generator,SlidingWindowSplitter


In [259]:
exp.plot_model(plot='train_test_split')


In [260]:
exp.compare_models(sort='rmse')


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,0.9835,0.9591,3.9127,4.8513,0.2670,0.2452,-1.1621,0.3330
ridge_cds_dt,Ridge w/ Cond. Deseasonalize & Detrending,1.0297,0.9952,4.1509,5.1056,0.2883,0.2569,-1.7169,0.3120
auto_arima,Auto ARIMA,1.0706,1.0029,4.2582,5.1323,0.2974,0.2508,-1.5251,11.7610
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,1.0206,1.0006,4.1371,5.1928,0.3005,0.2449,-2.5506,0.4830
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,1.0784,1.0313,4.3321,5.2983,0.3191,0.2556,-2.8095,0.3220
llar_cds_dt,Lasso Least Angular Regressor w/ Cond. Deseasonalize & Detrending,1.0784,1.0313,4.3321,5.2983,0.3191,0.2556,-2.8095,0.3250
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,1.0604,1.0280,4.2998,5.3183,0.3035,0.3039,-2.5011,0.3610
en_cds_dt,Elastic Net w/ Cond. Deseasonalize & Detrending,1.0717,1.0373,4.3061,5.3193,0.3121,0.2545,-2.5254,0.3200
dt_cds_dt,Decision Tree w/ Cond. Deseasonalize & Detrending,1.0859,1.0569,4.3236,5.4233,0.2852,0.2676,-1.1461,0.3370
croston,Croston,1.1174,1.0393,4.5681,5.4240,0.3231,0.2680,-1.8330,0.0170


BaseCdsDtForecaster(fe_target_rr=[WindowSummarizer(lag_feature={'lag': [24, 23,
                                                                        22, 21,
                                                                        20, 19,
                                                                        18, 17,
                                                                        16, 15,
                                                                        14, 13,
                                                                        12, 11,
                                                                        10, 9,
                                                                        8, 7, 6,
                                                                        5, 4, 3,
                                                                        2, 1]},
                                                   n_jobs=1)],
                    regressor=HuberRegressor(), sp=24, window_length=24)

In [324]:
ets = exp.create_model('huber_cds_dt')


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2024-01-06 09:00,0.9700,0.8222,3.3520,4.0181,0.1794,0.1821,0.1994
1,2024-01-11 19:00,1.1829,1.2907,3.5767,4.9467,0.1862,0.2134,-0.2009
2,2024-01-17 05:00,0.6915,0.7244,3.7239,4.7653,0.2792,0.2333,-1.8319
3,2024-01-22 15:00,1.3007,1.0940,6.1082,6.8222,0.6253,0.5281,-7.4278
4,2024-01-28 01:00,1.0868,1.1406,4.3861,5.4816,0.2573,0.2515,-1.4098
5,2024-02-02 11:00,1.1852,1.0998,4.7209,5.4531,0.2856,0.2423,-0.2207
6,2024-02-07 21:00,0.5043,0.4788,3.1982,3.8559,0.1665,0.1527,0.7040
7,2024-02-13 07:00,0.6782,0.6326,2.9022,3.6288,0.1656,0.1544,-0.2661
8,2024-02-18 17:00,1.3141,1.2583,3.8308,4.6793,0.2572,0.2301,0.0006
9,2024-02-24 03:00,0.9209,1.0492,3.3283,4.8626,0.2673,0.2636,-1.1677


In [262]:
tuned_ets = exp.tune_model(ets)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2024-01-06 09:00,1.0269,0.8747,3.5488,4.2749,0.1947,0.1918,0.0938
1,2024-01-11 19:00,1.1419,1.2999,3.4525,4.9820,0.1762,0.2064,-0.2181
2,2024-01-17 05:00,0.7758,0.7812,4.1775,5.1394,0.3150,0.2551,-2.2941
3,2024-01-22 15:00,1.5388,1.4416,7.2263,8.9900,0.7241,0.5127,-13.6349
4,2024-01-28 01:00,1.0787,1.1430,4.3536,5.4930,0.2528,0.2520,-1.4198
5,2024-02-02 11:00,1.3528,1.2487,5.3881,6.1913,0.3103,0.2676,-0.5736
6,2024-02-07 21:00,0.5354,0.4926,3.3957,3.9673,0.1856,0.1662,0.6866
7,2024-02-13 07:00,0.6957,0.6502,2.9769,3.7296,0.1692,0.1586,-0.3374
8,2024-02-18 17:00,1.3362,1.2470,3.8952,4.6372,0.2666,0.2349,0.0185
9,2024-02-24 03:00,0.8607,0.8772,3.1107,4.0652,0.2445,0.2517,-0.5150


Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   26.5s finished


In [331]:
holdout_pred = exp.predict_model(tuned_ets, verbose=True)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,HuberRegressor,0.6501,0.7089,2.8325,4.0475,0.1337,0.1483,0.0950


In [332]:
#predicting the future values 7 days
holdout_pred

,y_pred
2024-02-28 02:00,13.9641
2024-02-28 03:00,12.9636
2024-02-28 04:00,15.0006
2024-02-28 05:00,17.2675
2024-02-28 06:00,17.8028
2024-02-28 07:00,19.4853
2024-02-28 08:00,19.7591
2024-02-28 09:00,17.9006
2024-02-28 10:00,17.3357
2024-02-28 11:00,16.7169


In [334]:
final_model = exp.finalize_model(tuned_ets)


In [368]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Assuming your data is stored in a DataFrame called df_train
endog = df['PM25']  # Endogenous variable (target variable)
exog = df[['O3', 'WS', 'TEMP', 'RH', 'WD']]  # Exogenous variables

# Define and train the SARIMAX model
model = SARIMAX(endog, exog=exog, order=(1, 0, 1), seasonal_order=(1, 1, 1, 24))
result = model.fit()

# Forecasting
forecast_horizon = 24  # Forecast horizon of 24 hours
forecast = result.forecast(steps=forecast_horizon, exog=exog.iloc[-forecast_horizon:])

# Extract the last datetime value from the train.csv DataFrame
last_datetime = df.index[-1]

# Create a new datetime index for the forecast DataFrame
forecast_index = pd.date_range(start=last_datetime, periods=forecast_horizon + 1, freq='H')[1:]

# Create a DataFrame with forecasted values and the new datetime index
forecast_df = pd.DataFrame({'DATETIMEDATA': forecast_index, 'PM25': forecast})

forecast_df

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           10     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.71728D+00    |proj g|=  1.61845D-01


 This problem is unconstrained.



At iterate    5    f=  2.69496D+00    |proj g|=  2.33319D-01

At iterate   10    f=  2.63236D+00    |proj g|=  2.78912D-02

At iterate   15    f=  2.62549D+00    |proj g|=  6.10178D-02

At iterate   20    f=  2.61930D+00    |proj g|=  1.76111D-02

At iterate   25    f=  2.61897D+00    |proj g|=  1.33228D-02

At iterate   30    f=  2.61842D+00    |proj g|=  5.24540D-02

At iterate   35    f=  2.61819D+00    |proj g|=  1.82471D-03

At iterate   40    f=  2.61814D+00    |proj g|=  1.34068D-02

At iterate   45    f=  2.61810D+00    |proj g|=  8.03296D-03

At iterate   50    f=  2.61808D+00    |proj g|=  1.69998D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tn

,DATETIMEDATA,PM25
1416,1970-01-01 01:00:00.000001415,19.955712
1417,1970-01-01 02:00:00.000001415,18.270442
1418,1970-01-01 03:00:00.000001415,17.604323
1419,1970-01-01 04:00:00.000001415,16.057907
1420,1970-01-01 05:00:00.000001415,18.010508
1421,1970-01-01 06:00:00.000001415,19.302733
1422,1970-01-01 07:00:00.000001415,20.544571
1423,1970-01-01 08:00:00.000001415,21.374697
1424,1970-01-01 09:00:00.000001415,21.700418
1425,1970-01-01 10:00:00.000001415,19.214231


In [361]:
# Number of additional time steps to predict beyond the initial forecast horizon
additional_steps = 168  # For example, predicting 7 days ahead (24 hours * 7 days)

# Extend the forecast beyond the initial horizon
extended_forecast = result.get_forecast(steps=forecast_horizon + additional_steps, exog=exog.iloc[-(forecast_horizon + additional_steps):])

# Get the predicted values
predicted_values = extended_forecast.predicted_mean

# Extract the last timestamp from the train.csv data
last_timestamp = df['DATETIMEDATA'].iloc[-1]

# Generate a sequence of timestamps for the forecasted period
forecast_dates = pd.date_range(start=last_timestamp, periods=forecast_horizon + additional_steps + 1, freq='H')[1:]

# Create a DataFrame for the predicted values
predicted_df = pd.DataFrame({'DATETIMEDATA': forecast_dates, 'PM25_prediction': predicted_values})


predicted_df


,DATETIMEDATA,PM25_prediction
1416,2024-02-29 00:00:00,20.075080
1417,2024-02-29 01:00:00,18.998370
1418,2024-02-29 02:00:00,18.331131
1419,2024-02-29 03:00:00,16.703528
1420,2024-02-29 04:00:00,18.749276
...,...,...
1603,2024-03-07 19:00:00,18.607175
1604,2024-03-07 20:00:00,18.714532
1605,2024-03-07 21:00:00,18.501166
1606,2024-03-07 22:00:00,18.972411


In [349]:
#save to csv
predicted_values.to_csv('predict_hour.csv')

In [249]:
%pip install 'sktime'


77161.45s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Note: you may need to restart the kernel to use updated packages.
